# Decision Tree Method

In [1]:
import pandas as pd
import numpy as np

In [2]:
## load a pickle file
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")

print('train_df.shape: ', train_df.shape)
print('test_df.shape: ', test_df.shape)

train_df.head()

train_df.shape:  (1455563, 5)
test_df.shape:  (411972, 5)


,hashtag,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
5,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
6,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(train_df['text'])
vectorizer.get_feature_names_out()[100:110]

array(['asked', 'asking', 'ass', 'at', 'august', 'away', 'awesome',
       'awkward', 'baby', 'back'], dtype=object)

In [5]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopword = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hsiaoping.zhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
## baseline model : Random Forest
import pickle

tv = TfidfVectorizer(
            ngram_range = (1,1), # 一次提取多少詞為一個 feature 最多一次提取 1 個 最少 1 個
            max_features = 1000, # 找出最多出現的 1000 個詞,
            stop_words=stopword,
            token_pattern='(?u)\\b\\w\\w+\\b')

analyzer = tv.build_analyzer()
title_train = tv.fit_transform(train_df['text'])
pickle.dump(tv.vocabulary_, open("TFIDF_feature.pkl","wb"))

vocab_titrain = tv.get_feature_names()
dist = np.sum(title_train, axis=0)
checking_train = pd.DataFrame(dist,columns = vocab_titrain)
#checking_train.sort_values(by = 0, axis = 1, ascending = False)
checking_train

/Users/hsiaoping.zhang/tensorflow_macos_venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,00am,00pm,01,02,03,04,05,06,07,...,ya,yeah,year,years,yes,yesterday,yet,yo,young,youtube
0,953.381953,1344.144443,1102.270872,1303.748394,1188.238455,1146.585667,1260.96394,1307.702982,1236.456253,1168.40325,...,2191.254503,1991.565798,6439.713886,4142.643848,4288.068357,2367.405908,4087.937808,1156.374577,1491.770708,1747.869507


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

transformer = TfidfTransformer()
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("TFIDF_feature.pkl", "rb")))

# for a classificaiton problem, you need to provide both training & testing data
X = transformer.fit_transform(loaded_vec.fit_transform(train_df['text'])).toarray()
y = train_df['emotion']
print('finish')

finish


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

x_test = transformer.fit_transform(loaded_vec.fit_transform(test_df['text'])).toarray()
y_test = test_df['emotion']

## take a look at data dimension is a good habit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

X_train.shape:  (975227, 1000)
y_train.shape:  (975227,)
X_val.shape:  (480336, 1000)
y_val.shape:  (480336,)


In [ ]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=22)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_val_pred = DT_model.predict(X_val)

## so we get the pred result
y_val_pred[:10]

In [ ]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_val, y_pred=y_val_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('val accuracy: {}'.format(round(acc_val, 2)))

## precision, recall, f1-score,
from sklearn.metrics import classification_report

print(classification_report(y_true=y_val, y_pred=y_val_pred))

In [ ]:
# x_test = transformer.fit_transform(loaded_vec.fit_transform(test_df['text'])).toarray()
y_test_pred = DT_model.predict(X_test)
y_test_pred[:5]

In [ ]:
test_df['emotion'] = y_test_pred

ans_df = test_df[['tweet_id', 'emotion']]
ans_df = ans_df.rename(columns={"tweet_id": "id"})
# ans_df.to_csv('answer_decision_tree.csv', index=False)
print('finish')